# Memorization Metrics Analysis

This notebook analyzes memorization behaviors across different models, datasets, and normalization types. It leverages previously computed results (e.g., memorization_metrics.csv, features.csv) to explore:
- Correlation between mobility features and memorization (e.g., how radius of gyration or diversity relate to exposure).
- Comparative strength of different memorization types (e.g., substitute vs. shuffle).
- Cross-model comparison to assess which models are more prone to memorization.

The goal is to identify patterns, biases, or vulnerabilities in user memorization by location prediction models.

In [13]:
%pip install joypy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from scipy.stats import binned_statistic_2d
from scipy.ndimage import gaussian_filter
from collections import defaultdict
import numpy as np
from scipy.stats import gaussian_kde


In [57]:
DATASETS = {
    "ShenzhenUrban": {
        "type1": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShenzhenUrban/NormalizationType1/",
        "type2_home": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShenzhenUrban/NormalizationType2/Home/",
        "type2_work": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShenzhenUrban/NormalizationType2/Work/",
        "type3": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShenzhenUrban/NormalizationType3/",
    },
    "ShanghaiKaggle": {
        "type1": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShanghaiKaggle/NormalizationType1/",
        "type2_home": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShanghaiKaggle/NormalizationType2/Home/",
        "type2_work": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShanghaiKaggle/NormalizationType2/Work/",
        "type3": "/home/akouamdj/mobleak-datasets/PreprocessedData/2SplittedData/ShanghaiKaggle/NormalizationType3/",
    }
}

dataset_name = "ShanghaiKaggle"

#### Question 1: Is there any memorization?

In [44]:
model_name = "deepmove_simple"
type_name = "type1"

# Paths
ROOT = Path("results") / dataset_name / model_name / type_name
INPUT_FILE = ROOT / "memorization_metrics.csv"
PERPLEXITY_DIR = ROOT / "perplexity"
OUTPUT_DIR = Path("analysis_outputs") / dataset_name / "question1_memorization" / model_name / type_name
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load data
df = pd.read_csv(INPUT_FILE)

# Compute log2(N) exposure threshold dynamically from perplexity files
# logN_list = []
# for cluster_file in df["cluster_id"]:
#     perplexity_file = PERPLEXITY_DIR / cluster_file
#     if perplexity_file.exists():
#         try:
#             ref_df = pd.read_csv(perplexity_file)
#             N = len(ref_df)
#             logN_list.append(np.log2(N))
#         except Exception:
#             logN_list.append(np.nan)
#     else:
#         logN_list.append(np.nan)
# df["log2_N"] = logN_list
# df["exposure_gt_logN"] = df["exposure"] > df["log2_N"]

# Summary statistics
summary = {
    "num_trajectories": len(df),
    "gap_mean": df["gap"].mean(),
    "gap_median": df["gap"].median(),
    "percent_gap_below_zero": (df["gap"] < 0).mean() * 100,
    "exposure_mean": df["exposure"].mean(),
    "exposure_median": df["exposure"].median(),
    #"percent_exposure_above_logN": df["exposure_gt_logN"].mean() * 100,
    #"exposure_logN_threshold_mean": np.nanmean(df["log2_N"]),
    "percentile_mean": df["percentile"].mean(),
    "percent_percentile_below_0.1": (df["percentile"] < 0.1).mean() * 100
}

# Save summary
with open(OUTPUT_DIR / "memorization_summary.json", "w") as f:
    json.dump(summary, f, indent=4)

# --- Plotting ---
sns.set_style('whitegrid')
palette = sns.color_palette("deep")
ci = 95

# Exposure CDF
plt.figure(figsize=(4, 4))
#plot_ecdf_with_ci(df["exposure"], "Exposure", OUTPUT_DIR / "exposure_cdf.png")
sns.ecdfplot(df["exposure"], linewidth=2, label="Exposure", color=palette[0])
#avg_logN = np.nanmean(df["log2_N"])
#plt.axvline(avg_logN, color='red', linestyle='--', label=f"mean log₂(N) ≈ {avg_logN:.2f}")
plt.xlabel("Exposure", fontsize=14)
plt.ylabel("CDF", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "exposure_cdf.png", dpi=300)
plt.close()

# Gap CDF
plt.figure(figsize=(4, 4))
sns.ecdfplot(df["gap"], linewidth=2, label="Gap", color=palette[1])
plt.axvline(0, color='red', linestyle='--', label="Gap = 0")
plt.xlabel("Gap", fontsize=14)
plt.ylabel("CDF", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "gap_cdf.png", dpi=300)
plt.close()

# Percentile CDF
plt.figure(figsize=(4, 4))
sns.ecdfplot(df["percentile"], linewidth=2, label="Percentile", color=palette[2])
plt.axvline(0.1, color='red', linestyle='--', label="10% Percentile")
plt.xlabel("Percentile", fontsize=14)
plt.ylabel("CDF", fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "percentile_cdf.png", dpi=300)
plt.close()

print(summary)



# Learning curve plot
res_file = Path("results") / dataset_name / model_name / type_name / "model" / "res.txt"

# Try to read and parse the file
if res_file.exists():
    with open(res_file, "r") as f:
        res_data = json.load(f)

    metrics = res_data.get("metrics", {})
    train_loss = metrics.get("train_loss", [])
    valid_loss = metrics.get("valid_loss", [])
    accuracy = metrics.get("accuracy", [])

    epochs = list(range(1, len(train_loss) + 1))

    # Find best epoch by accuracy
    best_acc_epoch = int(np.argmax(accuracy)) + 1
    best_acc = accuracy[best_acc_epoch - 1]

    # Find epoch with minimum validation loss
    best_loss_epoch = int(np.argmin(valid_loss)) + 1
    best_loss = valid_loss[best_loss_epoch - 1]

    # Create the figure and twin axes
    fig, ax1 = plt.subplots(figsize=(5, 3.5))

    # Plot train and validation loss
    ax1.plot(epochs, train_loss, label="Train Loss", marker="o", alpha=0.6)
    ax1.plot(epochs, valid_loss, label="Validation Loss", marker="o", alpha=0.6)
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.tick_params(axis='y')

    # Mark best accuracy epoch
    #ax1.axvline(best_acc_epoch, color="red", linestyle="--", alpha=0.6)
    ax1.axvline(x=best_acc_epoch, color="red", linestyle="--", alpha=0.6, label=f"Best Acc (Epoch {best_acc_epoch})")

    # ax1.text(best_acc_epoch + 0.5, ax1.get_ylim()[1]*0.95,
    #          f"Best Acc (Epoch {best_acc_epoch})",
    #          color="red", fontsize=8)

    # Mark best validation loss epoch
    ax1.scatter([best_loss_epoch], [best_loss], color="green", s=40, label="Min Val Loss")
    # ax1.text(best_loss_epoch + 0.5, best_loss + 0.02,
    #          f"Min Val Loss (Epoch {best_loss_epoch})",
    #          color="green", fontsize=8)

    # Accuracy as secondary Y-axis
    ax2 = ax1.twinx()
    ax2.plot(epochs, accuracy, color="gray", linestyle="--", marker="x", alpha=0.5, label="Validation Accuracy")
    ax2.set_ylabel("Accuracy", color="gray")
    ax2.tick_params(axis='y', labelcolor="gray")

    # Combined legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    plt.legend(lines1 + lines2, labels1 + labels2, loc="center", fontsize=8)

    #plt.title(f"Learning Curve – {model_name} on {type_name}")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "learning_curve.png", dpi=300)
    plt.close()
else:
    print("res.txt not found.")




{'num_trajectories': 2000, 'gap_mean': np.float64(-1.0581904371491038), 'gap_median': np.float64(-1.162645), 'percent_gap_below_zero': np.float64(87.1), 'exposure_mean': np.float64(2.1439974578860097), 'exposure_median': np.float64(1.7858751946471525), 'percentile_mean': np.float64(0.3354867385139166), 'percent_percentile_below_0.1': np.float64(21.8)}


In [40]:
best_acc_epoch

11

#### Question 2: How do individual mobility behaviors correlate with memorization?
Assess how mobility features (e.g., rg, diversity) correlate with memorization (exposure, gap, etc.)

In [14]:
model_name = "deepmove_simple"
type_name = "type1"

# Paths
MEMO_PATH = Path("results") / dataset_name / model_name / type_name / "memorization_metrics.csv"
MOBILITY_PATH = Path(DATASETS[dataset_name][type_name]) / "mobility_characteristics.csv"
OUTPUT_DIR = Path("analysis_outputs") / dataset_name / "question2_mobility_correlation" / model_name / type_name
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load data
df_memo = pd.read_csv(MEMO_PATH)
df_mob = pd.read_csv(MOBILITY_PATH)

# Merge on tid
df = pd.merge(df_memo, df_mob, on="tid")

# Set seaborn style
sns.set_style("whitegrid")
palette = sns.color_palette("deep")
plt.rcParams["figure.figsize"] = (4, 3)

# === 1. CDFs of mobility features ===
mobility_metrics = ['diversity', 'rg_unique', 'stationarity', 'repetitiveness']

for i, metric in enumerate(mobility_metrics):
    plt.figure(figsize=(3, 3))
    sns.ecdfplot(df[metric], linewidth=2, color=palette[i])
    plt.xlabel(metric.replace('_', ' ').title(), fontsize=12)
    plt.ylabel("CDF", fontsize=12)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"{metric}_cdf.png", dpi=300)
    plt.close()

# Histogram of profiles
plt.figure(figsize=(3, 3))
sns.countplot(data=df, x="profile", palette="Set2")
plt.xlabel("Mobility Profile", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "profile_histogram.png", dpi=300)
plt.close()

# === 2. Heatmap: Percentile by rg and diversity quartiles ===
try:
    df["rg_q"] = pd.qcut(df["rg_unique"], 5, labels=[f"Q{i+1}" for i in range(5)])
    df["diversity_q"] = pd.qcut(df["diversity"], 5, labels=[f"Q{i+1}" for i in range(5)])

    heatmap_data = df.groupby(["diversity_q", "rg_q"])["percentile"].mean().unstack()

    plt.figure(figsize=(4, 3))
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu", 
                cbar_kws={"label": "Avg Memorization Percentile"})
    plt.xlabel("Radius of gyration Quartile (Q1 = Low)", fontsize=12)
    plt.ylabel("Diversity Quartile (Q1 = Low)", fontsize=12)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "heatmap_percentile_by_rg_diversity.png", dpi=300)
    plt.close()

except Exception as e:
    print("Heatmap plot failed:", e)


# === 3. Contour plot: Repetitiveness vs Stationarity ===
try:
    x = df["stationarity"]
    y = df["repetitiveness"]
    z = df["percentile"]

    # Define grid
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate z values on the grid
    from scipy.interpolate import griddata
    zi = griddata((x, y), z, (xi, yi), method='linear')

    # Plot contour
    plt.figure(figsize=(4, 3))
    cp = plt.contourf(xi, yi, zi, levels=20, cmap="RdYlBu")
    plt.colorbar(cp, label="Avg Memorization Percentile")
    plt.xlabel("Stationarity")
    plt.ylabel("Repetitiveness")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "contour_percentile_by_stationarity_repetitiveness.png", dpi=300)
    plt.close()
except Exception as e:
    print("Contour plot failed:", e)

# === 4. Ridge Plot: Percentile distribution by mobility profile ===
try:
    import joypy

    plt.figure(figsize=(3, 3))
    fig, axes = joypy.joyplot(
        df, 
        by="profile", 
        column="percentile", 
        kind="kde", 
        fill=True, 
        colormap=plt.cm.Blues,
        linewidth=1,
        fade=True,
        alpha=0.9,
        figsize=(3,3)
    )
    plt.xlabel("Memorization Percentile")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "ridge_percentile_by_profile.png", dpi=300)
    plt.close()
except Exception as e:
    print("Ridge plot failed:", e)

    
# CDF by profile
plt.figure(figsize=(3, 3))
for profile in df["profile"].unique():
    sns.ecdfplot(df[df["profile"] == profile]["percentile"], label=profile, linewidth=2)
plt.xlabel("Memorization Percentile", fontsize=12)
plt.ylabel("CDF", fontsize=12)
plt.legend(title="Profile")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "percentile_by_profile.png", dpi=300)
plt.close()


/tmp/ipykernel_1526416/3712378792.py:36: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x="profile", palette="Set2")
/tmp/ipykernel_1526416/3712378792.py:48: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  heatmap_data = df.groupby(["diversity_q", "rg_q"])["percentile"].mean().unstack()


<Figure size 300x300 with 0 Axes>

#### Question 3: Are models more prone to certain memorization types?

In [59]:
# === Config ===
model_name = "deepmove_simple"
#model_name = "markov"
type_names = ["type1", "type2_home", "type2_work", "type3"]
metrics = ["exposure", "gap", "percentile"]

INPUT_ROOT = Path("results") / dataset_name / model_name
OUTPUT_DIR = Path("analysis_outputs") / dataset_name / "question3_memorization_types" / model_name
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Load and tag data for all memorization types ---
df_all = []
for type_name in type_names:
    path = INPUT_ROOT / type_name / "memorization_metrics.csv"
    if path.exists():
        df = pd.read_csv(path)
        df["type"] = type_name
        df_all.append(df)
df_all = pd.concat(df_all, ignore_index=True)

# === 1. CDF plots (Kernel Estimation) for each memorization metric by type ===
for metric in metrics:
    plt.figure(figsize=(3, 3))
    for mem_type in df_all["type"].unique():
        data = df_all[df_all["type"] == mem_type][metric].dropna()
        if len(data) < 2:
            continue  # Skip if not enough data

        # Kernel Density Estimation
        kde = gaussian_kde(data)
        x_vals = np.linspace(data.min(), data.max(), 500)
        pdf = kde(x_vals)
        cdf = np.cumsum(pdf)
        cdf = cdf / cdf[-1]  # Normalize

        plt.plot(x_vals, cdf, label=mem_type, linewidth=2)

    plt.xlabel(metric.title(), fontsize=12)
    plt.ylabel("CDF", fontsize=12)
    plt.legend(fontsize=7)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"cdf_{metric}_by_type.png", dpi=300)
    plt.close()

# === 2. Stacked bar chart of mobility profiles per memorization type ===
profile_counts_by_type = defaultdict(lambda: defaultdict(int))
for type_name in type_names:
    mobility_file = Path(DATASETS[dataset_name][type_name]) / "mobility_characteristics.csv"
    # Load and join
    df_mob = pd.read_csv(mobility_file)[["tid", "profile"]]
    counts = df_mob["profile"].value_counts()
    for profile, count in counts.items():
        profile_counts_by_type[type_name][profile] += count
df_counts = pd.DataFrame(profile_counts_by_type).fillna(0).T
df_props = df_counts.div(df_counts.sum(axis=1), axis=0)
plt.figure(figsize=(3, 3))
ax = df_props[["routiner", "regular", "scouter"]].plot(
    kind="bar", stacked=True, colormap="Set3", width=0.7
)
plt.ylabel("", fontsize=12)
plt.xlabel("", fontsize=12)
plt.xticks(rotation=15, fontsize=8)  # Slight rotation for clarity
plt.legend(
    loc="lower center", 
    bbox_to_anchor=(0.5, 1.0),
    ncol=3,
    fontsize=8,
    frameon=False
)
# Annotate percentages
for i, row in enumerate(df_props[["routiner", "regular", "scouter"]].values):
    cum_height = 0
    for j, val in enumerate(row):
        if val > 0.01:  # only annotate if it's not too small
            ax.text(
                i, cum_height + val / 2,
                f"{val*100:.1f}%",
                ha='center', va='center', fontsize=7, color="black"
            )
        cum_height += val

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "stacked_profiles_by_type.png", dpi=300)
plt.close()


# === 3. Boxplots for gap, exposure, percentile by memorization type ===
modes = {
    "": "all",
    "_substitute": "substitute",
    "_stationary": "stationary",
    "_shuffle": "shuffle"
}
df_type3 = df_all[df_all["type"] == "type3"]
for metric in metrics:
    data = []
    for suffix, label in modes.items():
        column = f"{metric}{suffix}"
        if column in df_type3.columns:
            for value in df_type3[column].dropna():
                data.append({"mode": label, metric: value})

    df_metric = pd.DataFrame(data)
    plt.figure(figsize=(4, 3))
    sns.boxplot(data=df_metric, x="mode", y=metric, palette="deep")
    plt.xlabel("", fontsize=12)
    plt.xticks(fontsize=9)
    plt.ylabel(metric.title(), fontsize=12)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"boxplot_type3_{metric}_by_mode.png", dpi=300)
    plt.close()

# === 4. Radar Plot: average value of metrics per memorization type ===

# Compute average metrics per type
avg_metrics = df_all.groupby("type")[metrics].median()
avg_metrics["gap"] = avg_metrics["gap"].abs()  # Use absolute gap for radar plot

# Radar plot setup
labels = metrics
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # repeat first angle

fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))
for i, (type_name, row) in enumerate(avg_metrics.iterrows()):
    values = row.tolist()
    values += values[:1]  # repeat first value to close the circle
    ax.plot(angles, values, label=type_name, linewidth=2)
    ax.fill(angles, values, alpha=0.2)

ax.set_thetagrids(np.degrees(angles[:-1]), labels)
#plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
plt.legend(fontsize=7)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "radar_median_metrics_by_type.png", dpi=300)
plt.close()


# === 5. Type 3 memorization per hour of the day ===
INPUT_FILE = Path("results") / dataset_name / model_name / "type3" / "memorization_metrics_per_window.csv"
df_win = pd.read_csv(INPUT_FILE)
df_win["hour_bin"] = df_win["hour_of_day"] % 24
df_win["hour_bin"] = pd.cut(
    df_win["hour_bin"],
    bins=[0, 4, 8, 12, 16, 20, 24],
    right=False,
    labels=["[0–3]", "[4–7]", "[8–11]", "[12–15]", "[16–19]", "[20–23]"]
)
for metric in metrics:
    plt.figure(figsize=(4, 3))
    sns.violinplot(
        data=df_win,
        x="hour_bin",
        y=metric,
        inner="quartile",
        scale="width",
        cut=0,
        linewidth=0.8,
        palette="viridis"
    )
    plt.xlabel("Hour of day", fontsize=12)
    plt.ylabel(metric.title(), fontsize=12)
    #plt.title(f"Distribution of {metric.title()} by Hour of Day Period", fontsize=13)
    plt.xticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"violin_{metric}_by_hour_period.png", dpi=300)
    plt.close()

/tmp/ipykernel_1526416/963178238.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_metric, x="mode", y=metric, palette="deep")
/tmp/ipykernel_1526416/963178238.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_metric, x="mode", y=metric, palette="deep")
/tmp/ipykernel_1526416/963178238.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_metric, x="mode", y=metric, palette="deep")
/tmp/ipykernel_1526416/963178238.py:152: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be remo

<Figure size 300x300 with 0 Axes>

#### Question 4: Are some models more prone to memorization than others?

In [ ]:

question = "question4_model_comparison"
type_name = "type1"
metrics = ["exposure", "gap", "percentile"]

# Directory where all results are saved
RESULTS_DIR = Path("results") / dataset_name
OUTPUT_DIR = Path("analysis_outputs") / dataset_name / question
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# === Load and merge all models for the selected type ===
dfs = []
model_names = []
carlini_values = {}

for model_dir in RESULTS_DIR.iterdir():
    model_name = model_dir.name
    model_names.append(model_name)

    mem_file = model_dir / type_name / "memorization_metrics.csv"
    carlini_file = model_dir / "canaries" / "carlini_exposure.json"

    if mem_file.exists():
        df = pd.read_csv(mem_file)
        df["model"] = model_name
        df["type"] = type_name
        dfs.append(df)

        # Load Carlini exposure value
        if carlini_file.exists():
            with open(carlini_file, "r") as f:
                data = json.load(f)
                carlini_values[model_name] = {
                    "exposure": data["exposure"],
                    "gap": data["gap"],
                    "percentile": data["percentile"]
                }

df_all = pd.concat(dfs, ignore_index=True)

# === Ridge Plots ===
sns.set_style("whitegrid")
for metric in metrics:
    # Sort models by average metric value (optional, for better order)
    model_order = df_all.groupby("model")[metric].mean().sort_values().index.tolist()
    n_models = len(model_order)
    height_per_facet = 0.7
    total_height = n_models * height_per_facet

    g = sns.FacetGrid(
        df_all, 
        row="model", 
        hue="model", 
        aspect=4, 
        height=height_per_facet, 
        row_order=model_order,
        sharex=True, 
        sharey=False,
        palette="deep"
    )
    g.map(sns.kdeplot, metric, fill=True, alpha=0.6, linewidth=1.5)
    g.set_titles(row_template="{row_name}", size=6)
    g.set_xlabels(metric.title(), fontsize=12)
    g.set_ylabels("")
    g.set(yticks=[])  # Remove density ticks
    g.despine(left=True)

    plt.subplots_adjust(top=0.95, hspace=0.3)
    g.savefig(OUTPUT_DIR / f"ridgeplot_{metric}_by_model.png", dpi=300)
    plt.close()
# sns.set_style("whitegrid")
# n_models = len(model_names)
# n_cols = 3
# n_rows = int(np.ceil(n_models / n_cols))
# for metric in metrics:
#     g = sns.FacetGrid(
#         df_all, 
#         col="model", 
#         col_wrap=n_cols,
#         aspect=1.5, 
#         height=2,
#         sharex=True, 
#         sharey=False,
#         palette="deep"
#     )
#     g.map(sns.kdeplot, metric, fill=True, alpha=0.6, linewidth=1.5)

#     g.set_titles(col_template="{col_name}", size=11)
#     g.set_xlabels(metric.title(), fontsize=12)
#     g.set_ylabels("")
#     g.set(yticks=[])
#     g.despine(left=True)

#     plt.subplots_adjust(top=0.92, hspace=0.4, wspace=0.3)
#     g.savefig(OUTPUT_DIR / f"ridgeplot_{metric}_by_model_grid.png", dpi=300)
#     plt.close()

# === Ridge Plots with Carlini Lines ===
sns.set_style("whitegrid")
for metric in metrics:
    model_order = df_all.groupby("model")[metric].mean().sort_values().index.tolist()
    n_models = len(model_order)
    height_per_facet = 0.7
    total_height = n_models * height_per_facet

    g = sns.FacetGrid(
        df_all,
        row="model",
        hue="model",
        aspect=4,
        height=height_per_facet,
        row_order=model_order,
        sharex=True,
        sharey=False,
        palette="deep"
    )

    g.map(sns.kdeplot, metric, fill=True, alpha=0.6, linewidth=1.5)

    # Add Carlini line per subplot
    for ax, model in zip(g.axes.flat, model_order):
        if model in carlini_values:
            x_val = carlini_values[model][metric]
            ax.axvline(x=x_val, color="black", linestyle="--", linewidth=1)
            ax.text(
                x_val, ax.get_ylim()[1] * 0.8,  # Position text near the top
                "Carlini", rotation=90, color="black", fontsize=5,
                ha="right", va="center"
            )

    g.set_titles(row_template="{row_name}", size=6)
    g.set_xlabels(metric.title(), fontsize=12)
    g.set_ylabels("")
    g.set(yticks=[])
    g.despine(left=True)

    plt.subplots_adjust(top=0.95, hspace=0.3)
    g.savefig(OUTPUT_DIR / f"ridgeplot_{metric}_by_model_with_carlini.png", dpi=300)
    plt.close()


#### Question 5: Assessing the Impact of Memorization on Cluster-Level Generalization

In [17]:
# Define model and paths
model_name = "deepmove_simple"  # Change this manually for different models
#model_name = "lstpm"  
type_name = "type1"

BASE_PATH = Path("results") / dataset_name / model_name / type_name
MEM_FILE = BASE_PATH / "memorization_metrics.csv"
TEST_PATH = BASE_PATH / "test"

OUTPUT_DIR = Path("analysis_outputs") / dataset_name / "question5_accuracy_vs_memorization" / model_name
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load memorization file
df_mem = pd.read_csv(MEM_FILE)
df_mem["cluster_id"] = df_mem["cluster_id"].astype(str)

# Initialize
accuracy_metrics = ["top-1", "top-5", "top-10"]
cluster_accuracy = {}

# Extract accuracy per cluster from test files
for cid in df_mem["cluster_id"].unique():
    cid_clean = cid.replace(".csv", "")
    test_file = TEST_PATH / f"{cid_clean}_topk.csv"
    if not test_file.exists():
        print(f"Test file for cluster {cid} not found, skipping.")
        continue
    df_test = pd.read_csv(test_file)
    cluster_accuracy[cid] = df_test[accuracy_metrics].mean().to_dict()

# Convert accuracy dict to DataFrame
df_acc = pd.DataFrame(cluster_accuracy).T
df_acc.index.name = "cluster_id"
df_acc.reset_index(inplace=True)

# Merge with memorization metrics
df_merged = df_mem.merge(df_acc, on="cluster_id", how="inner")

# Plot 1: Scatterplot with trendline per memorization metric vs top-1 accuracy
mem_metrics = ["exposure", "gap", "percentile"]
for metric in mem_metrics:
    plt.figure(figsize=(3, 3))
    sns.regplot(data=df_merged, x=metric, y="top-1", scatter_kws={"s": 10, "alpha": 0.5}, line_kws={"color": "red"})
    plt.xlabel(metric.title())
    plt.ylabel("Top-1 Accuracy")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"scatter_{metric}_vs_top1.png", dpi=300)
    plt.close()

# Plot 2: Boxplots of memorization metrics grouped by accuracy levels (binned top-k)
quartiles = 5
for metric in mem_metrics:
    df_temp = df_merged.copy()
    # Bin the memorization metric into quartiles (Q1–Q4)
    df_temp[f"{metric}_quartile"] = pd.qcut(df_temp[metric], quartiles, labels=[f"Q{i+1}" for i in range(quartiles)])
    # Melt accuracy columns to long format
    df_long = df_temp.melt(
        id_vars=["tid", "cluster_id", f"{metric}_quartile"],
        value_vars=accuracy_metrics,
        var_name="topk",
        value_name="accuracy"
    )
    # Plot
    plt.figure(figsize=(3, 3))
    sns.boxplot(
        data=df_long,
        x=f"{metric}_quartile",
        y="accuracy",
        hue="topk",
        palette="pastel"
    )
    #plt.xlabel(f"{metric.title()} Quartile", fontsize=12)
    plt.xlabel(f"{metric.title()} Quartile (Q1 = Low)", fontsize=10)
    plt.ylabel("Top-k Accuracy", fontsize=12)
    plt.legend(
        loc="upper center", 
        bbox_to_anchor=(0.5, 1.1), 
        ncol=len(accuracy_metrics),
        fontsize=7,
        title_fontsize=10,
        frameon=False
    )
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"boxplot_accuracy_vs_{metric}_quartile.png", dpi=300)
    plt.close()


Test file for cluster cluster_2589456_2589552.csv not found, skipping.
Test file for cluster cluster_1452528_1452624.csv not found, skipping.
Test file for cluster cluster_1733712_1733808.csv not found, skipping.
Test file for cluster cluster_2527248_2527344.csv not found, skipping.
Test file for cluster cluster_3716304_3716400.csv not found, skipping.
Test file for cluster cluster_2536944_2537040.csv not found, skipping.
Test file for cluster cluster_1925472_1925568.csv not found, skipping.
Test file for cluster cluster_89136_89232.csv not found, skipping.
Test file for cluster cluster_21696_21792.csv not found, skipping.
Test file for cluster cluster_732480_732576.csv not found, skipping.
Test file for cluster cluster_483168_483264.csv not found, skipping.
Test file for cluster cluster_2627808_2627904.csv not found, skipping.
Test file for cluster cluster_482736_482832.csv not found, skipping.
Test file for cluster cluster_3417072_3417168.csv not found, skipping.
Test file for cluste

KeyError: 'top-1'

<Figure size 300x300 with 0 Axes>